In [1]:
!conda list

# packages in environment at /Users/lovkush/Desktop/Folder/Coding/learning/conda-env:
#
# Name                    Version                   Build  Channel
_py-xgboost-mutex         2.0                       cpu_0    conda-forge
alabaster                 0.7.12                     py_0  
applaunchservices         0.2.1                      py_0  
appnope                   0.1.0                    py38_0  
argh                      0.26.2                   py38_0  
astroid                   2.4.0                    py38_0  
atomicwrites              1.4.0                      py_0  
attrs                     19.3.0                     py_0  
autopep8                  1.4.4                      py_0  
babel                     2.8.0                      py_0  
backcall                  0.1.0                    py38_0  
beautifulsoup4            4.9.1                    py38_0    anaconda
blas                      1.0                         mkl  
bleach                    3.1.4           

In [2]:
import requests
from bs4 import BeautifulSoup
from IPython.core.display import HTML
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA

In [3]:
urls = ['https://en.wikipedia.org/wiki/Official_Women%27s_Squash_World_Ranking',
    'https://en.wikipedia.org/wiki/Official_Men%27s_Squash_World_Ranking']

In [4]:
def is_not_numeric(s):
    try:
        float(s)
    except ValueError:
        return True
    else:
        return False

def table_to_pandas(table):
    rows = table.find_all('tr')
    headers = [col.text.replace('\n', '') for col in rows[0].find_all('th')[1:]]
    
    n_cols = len(rows[-1].find_all('td'))    
    
    data = [
            [col.text.replace('\n','')
             for col in row.find_all('td')[1:]
             if is_not_numeric(col.text.replace('\n',''))
            ]
            for row in rows[1:]
           ]
    
    return pd.DataFrame(data, columns = headers, index = range(1,11))

def url_to_pandas(url):
    """
    headers contains id needed to cut html into two pieces
    """
    html = requests.get(url).text
    start = html.find('id="Year_end_world_top_10_players')
    end = html.find('id="Year-end_number_1')
    tables = BeautifulSoup(html[start:end], 'html.parser').find_all('table')

    df = pd.concat([table_to_pandas(t) for t in tables], axis=1)
    df_stack = df.stack().reset_index()
    df_stack.columns = ['rank', 'year', 'player']
    
    return df_stack

def player_summaries(df):
#     years_in_top10 = df.player.value_counts()
    players = ( df.groupby('player')
               .agg({'rank': [np.mean, 'count', np.min, np.max],
                     'year': [np.min, np.max]
                    })
              )

    players.columns = ['average_rank', 'years_in_top10','best_rank', 'worst_rank',
                       'earliest_year','latest_year']
    players.sort_values(by = ['average_rank'], inplace = True)
    
    return players
    

In [5]:
df_m = url_to_pandas(urls[1])

ValueError: 3 columns passed, passed data had 5 columns

In [4]:
df_f, df_m = [url_to_pandas(urls[i]) for i in range(2)]

In [5]:
players_f = player_summaries(df_f)
players_m = player_summaries(df_m)

In [6]:
players_f

,average_rank,years_in_top10,best_rank,worst_rank,earliest_year,latest_year
player,,,,,,
Michelle Martin,1.400000,5,1,2,1994,1998
Sarah Fitz-Gerald,2.333333,9,1,5,1994,2002
Nicol David,2.357143,14,1,6,2004,2017
Raneem El Weleily,2.444444,9,1,7,2011,2019
Leilani Rorani,2.750000,4,1,7,1998,2001
Nour El Sherbini,3.142857,7,1,6,2012,2019
Rachael Grinham,3.909091,11,1,8,2001,2011
Carol Owens,4.100000,10,1,8,1994,2003
Laura Massaro,4.111111,9,2,9,2010,2018


In [7]:
players_m

,average_rank,years_in_top10,best_rank,worst_rank,earliest_year,latest_year
player,,,,,,
Peter Nicol,2.400000,10,1,8,1996,2005
Ali Farag,3.250000,4,1,7,2016,2019
Jansher Khan,3.333333,3,1,8,1996,1998
Jonathon Power,3.666667,9,1,9,1997,2005
Mohamed El Shorbagy,3.700000,10,1,10,2010,2019
Ramy Ashour,3.909091,11,1,7,2006,2016
Grégory Gaultier,4.000000,15,1,10,2003,2018
Ahmed Barada,4.250000,4,2,7,1997,2000
Rodney Eyles,4.333333,3,2,7,1996,1998


# visuals and clustering and dimensionality reduction

In [1]:
players_m

NameError: name 'players_m' is not defined